In [1]:
!pip install flask nest_asyncio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from flask import Flask, render_template_string, request
import nest_asyncio
import threading

# Allow Flask to run within Jupyter Notebook
nest_asyncio.apply()

# Define the Flask app
app = Flask(__name__)

# Exchange rates (base: USD)
EXCHANGE_RATES = {
    'USD': 1.0,
    'EUR': 0.93,
    'INR': 83.2,
    'GBP': 0.78,
    'JPY': 141.1
}

# HTML template as a string
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Currency Converter</title>
    <style>
        body { font-family: Arial; background-color: #f0f0f0; text-align: center; padding-top: 50px; }
        input, select, button { padding: 10px; margin: 10px; width: 220px; }
        button { background-color: #008CBA; color: white; border: none; }
    </style>
</head>
<body>
    <h1>Currency Converter</h1>
    <form method="POST">
        <input type="text" name="amount" placeholder="Enter amount" required><br>
        <select name="from_currency">
            {% for currency in currencies %}
                <option value="{{ currency }}">{{ currency }}</option>
            {% endfor %}
        </select>
        to
        <select name="to_currency">
            {% for currency in currencies %}
                <option value="{{ currency }}">{{ currency }}</option>
            {% endfor %}
        </select><br>
        <button type="submit">Convert</button>
    </form>

    {% if result %}
        <h2>{{ result }}</h2>
    {% elif error %}
        <h2 style="color:red;">{{ error }}</h2>
    {% endif %}
</body>
</html>
"""

# Route handler
@app.route('/', methods=['GET', 'POST'])
def index():
    result = None
    error = None
    currencies = EXCHANGE_RATES.keys()

    if request.method == 'POST':
        try:
            amount = float(request.form['amount'])
            from_currency = request.form['from_currency']
            to_currency = request.form['to_currency']

            if from_currency not in EXCHANGE_RATES or to_currency not in EXCHANGE_RATES:
                error = "Invalid currency selection."
            else:
                usd_amount = amount / EXCHANGE_RATES[from_currency]
                converted = usd_amount * EXCHANGE_RATES[to_currency]
                result = f"{amount} {from_currency} = {round(converted, 2)} {to_currency}"
        except ValueError:
            error = "Please enter a valid number."

    return render_template_string(HTML_TEMPLATE, currencies=currencies, result=result, error=error)

# Run Flask app in separate thread so Jupyter doesn’t block
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [08/Aug/2025 21:45:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2025 21:45:01] "GET /favicon.ico HTTP/1.1" 404 -
